In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import SimpleImputer
import numpy as np
import re

# 데이터 로드
df = pd.read_csv('../data/Clean_Red_data.csv')

# 필요한 열만 선택
features = ['wine_name', 'body', 'texture', 'sweetness', 'acidity', 'flavor1', 'flavor2', 'flavor3']
df_features = df[features]

# 숫자형과 문자열형 열을 구분합니다.
num_features = ['body', 'texture', 'sweetness', 'acidity']
cat_features = ['flavor1', 'flavor2', 'flavor3']

# 숫자형과 문자열형 열을 위한 변환기 정의
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # 결측값 처리
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # 결측값 처리
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # 밀집 행렬로 변환
])

# 열을 변환할 수 있는 ColumnTransformer 정의
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ]
)

# 전체 데이터 전처리
X = preprocessor.fit_transform(df_features.drop('wine_name', axis=1))

# NearestNeighbors 모델 생성 (n_neighbors를 10으로 설정)
nbrs = NearestNeighbors(n_neighbors=10, algorithm='auto').fit(X)

def find_similar_wine(wine_name):
    # 입력 와인의 특성값을 찾습니다.
    input_wine = df[df['wine_name'] == wine_name]
    
    if input_wine.empty:
        return "해당 와인을 찾을 수 없습니다."
    
    input_wine_features = input_wine[features[1:]]
    
    # 입력 와인의 특성값을 전처리합니다.
    input_wine_features_transformed = preprocessor.transform(input_wine_features)
    
    # 가장 유사한 와인을 찾습니다.
    distances, indices = nbrs.kneighbors(input_wine_features_transformed)
    
    # 유사한 와인 목록을 준비합니다.
    similar_wines = []
    for i in range(len(indices[0])):
        wine_index = indices[0][i]
        if wine_index == df.index[df['wine_name'] == wine_name][0]:
            continue  # 자기 자신을 제외
        # 유사도 점수를 거리의 역수로 계산합니다.
        similarity_score = 1 / (distances[0][i] + 1e-5)  # 1e-5는 0으로 나누는 것을 방지합니다.
        similar_wines.append((df.iloc[wine_index]['wine_name'], similarity_score, distances[0][i]))

    # 유사도 순서대로 정렬
    similar_wines.sort(key=lambda x: x[1], reverse=True)
    
    # 와인 이름에서 연도를 제거하고, 기본 이름을 기준으로 가장 유사한 결과만 남기기
    def remove_year(wine_name):
        return re.sub(r'\b\d{4}\b', '', wine_name).strip()
    
    # 기본 이름을 기준으로 가장 유사한 결과만 남기기
    filtered_wines = {}
    for wine_name, score, distance in similar_wines:
        base_name = remove_year(wine_name)
        if base_name not in filtered_wines:
            filtered_wines[base_name] = (wine_name, score, distance)
        else:
            # 기존 와인과 비교하여 거리가 더 가까운 경우 업데이트
            if distance < filtered_wines[base_name][2]:
                filtered_wines[base_name] = (wine_name, score, distance)

    # 결과를 리스트로 변환하고 유사도 순서대로 정렬
    top_similar_wines = sorted(filtered_wines.values(), key=lambda x: x[1], reverse=True)[:10]
    
    return top_similar_wines

# 사용 예
wine_to_search = 'Pinot Noir 2021'
top_similar_wines = find_similar_wine(wine_to_search)
print(f"가장 유사한 와인들:")
for wine, similarity_score, distance in top_similar_wines:
    print(f"와인: {wine}, 유사성 점수: {similarity_score:.4f}, 거리: {distance:.4f}")


가장 유사한 와인들:
와인: Pinot Noir 2005, 유사성 점수: 99407.4852, 거리: 0.0000
와인: Finla Mor Pinot Noir 2022, 유사성 점수: 3.0192, 거리: 0.3312
와인: Evenstad Reserve Pinot Noir 2006, 유사성 점수: 0.7046, 거리: 1.4192
와인: Bella Reserve 2021, 유사성 점수: 0.6858, 거리: 1.4582
